In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
import random
import string
import json
from peft import LoraConfig, get_peft_model

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device : {device}")

Using device : cuda


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
for param in model.parameters():
  param.requires_grad = False
model.lm_head.weight.requires_grad = True

In [ ]:
for block in model.decoder.block[-2:]:
    for p in block.parameters():
        p.requires_grad = True

In [ ]:
class Ds(Dataset):
  def __init__(self, path):
    self.data = pd.read_json("cleaner-dataset.jsonl", lines=True) # Have the fileName Here that i used

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    src = tokenizer(self.data["input"][idx], truncation=True, padding="max_length", max_length=580, return_tensors="pt")
    trg = tokenizer(self.data["output"][idx], truncation=True, padding="max_length", max_length=580, return_tensors="pt")
    return src["input_ids"].squeeze(0), src["attention_mask"].squeeze(0), trg["input_ids"].squeeze(0)

dataset = Ds("cleaner-dataset.jsonl")

In [ ]:
trainDataLoader = DataLoader(
    dataset,
    6,
    True,
    num_workers = 2,
    pin_memory = True

)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), weight_decay=0.001)

In [ ]:
epochs = 10
for epoch in range(epochs):
  totalLoss = 0
  for src, mask, trg in trainDataLoader:
    optimizer.zero_grad()
    src, mask, trg = src.to(device), mask.to(device), trg.to(device)
    trg[trg == tokenizer.pad_token_id] = -100
    out = model(input_ids=src, attention_mask=mask, labels = trg)
    loss = out.loss
    totalLoss += loss.item()
    loss.backward()
    optimizer.step()

  print(f"The loss is : {totalLoss/ len(trainDataLoader)}")



The loss is : 1.0787503497273314
The loss is : 0.6165913647296382


KeyboardInterrupt: 

In [ ]:
def chat(text):
  data = tokenizer(text, return_tensors="pt").to(device)
  out = model.generate(**data,
                       max_length=200,
                       do_sample = True,
                       repetition_penalty = 1.2,
                       temperature = 0.8,
                       top_p = 0.9,
                       )
  print(text)
  print()
  print("The question-answer pairs are as follows:")
  print()
  print(tokenizer.decode(out[0]))

In [ ]:

chat("Helping each other is a good method of learning about life. The unity is what makes human superior overall.")


Helping each other is a good method of learning about life. The unity is what makes human superior overall.

The question-answer pairs are as follows:

<pad> Q1: Why is helping others beneficial? A1: They are better able to cope with stress and anxiety. Q2: What happens to human beings when they help each other? A2: Human beings are better able to cope with stress and anxiety.</s>
